In [12]:
from UCLSE.environment import Market_session, yamlLoad
from UCLSE.test.utils import identical_replay_vars,side_by_side_period_by_period_difference_checker
from UCLSE.exchange import Exchange
import copy

I have extensively rewritten the exchange.py file to account for variable quantity orders. The main complication of this that any new order can cause a greater number of outcomes. Consider a new bid. It can have the following effects:

- 0 Add itself to the LOB and do nothing
- 1 Bid exactly fills a best bid ask, two orders are deleted off LOB and two traders informed.
- 2a Bid completely fills with any number of ask orders at any number of prices. Any number of orders are deleted and any number of traders are informed.
- 2a As above, but only partially fill an existing ask on the LOB. Ask order owner needs to be made aware of order change.
- 3 Bid Partially filled - As above but bid owner needs to be made aware of change. 
<br>
There are also bookkeeping considerations which I haven't considered yet. 

But, to check the ammendments haven't messed up the original calculations, we should run parallel experiments using the old and new exchange methods to ensure that they result in the same thing. 

In [13]:
import os
pa=os.getcwd()
config_name='config\\mkt_cfg.yml'
config_path=os.path.join(pa,config_name)
environ_dic=yamlLoad(config_path)

#just updates the tradefile every time there is a trade
environ_dic['dump_each_trade']=True
environ_dic['verbose']=False
environ_dic['end_time']=50

I have add process_order as an attribute of the session object, to allow selection of which process order mechanism we use. It will default to the old one.

First check that the new method works. Begin by instantiating a Market_session and then copying it exactly.

In [14]:
import copy
sess=Market_session(**environ_dic)
sess.process_order=sess.exchange.process_order3w
sess1=copy.deepcopy(sess)


Then we can run the first simulation, store the events and push those events into the second simulation session to make sure that the end result is the same.

In [15]:
sess.simulate(sess.trade_stats_df3,recording=True)
sess1.simulate(sess1.trade_stats_df3,replay_vars=sess.replay_vars,recording=True)
(sess1.df.fillna(0)==sess.df.fillna(0)).all() #note that the best_bid or best_bid do not always give a True value. Because nan!=nan.

Bid  leg 0  lifts best  Ask 98
counterparty S05 price 95
[{'p2_qid': 6.0, 'party1': 'S05', 'qty': 1, 'party2': 'B00', 'type': 'Trade', 'p1_qid': 5, 'time': 1.974999999999995, 'price': 95}]
Ask  leg 0  lifts best  Bid 101
counterparty B37 price 105
[{'p2_qid': 29.0, 'party1': 'B37', 'qty': 1, 'party2': 'S24', 'type': 'Trade', 'p1_qid': 15, 'time': 4.700000000000033, 'price': 105}]
Ask  leg 0  lifts best  Bid 105
counterparty B34 price 105
[{'p2_qid': 35.0, 'party1': 'B34', 'qty': 1, 'party2': 'S26', 'type': 'Trade', 'p1_qid': 19, 'time': 5.125000000000039, 'price': 105}]
Ask  leg 0  lifts best  Bid 95
counterparty B03 price 105
[{'p2_qid': 49.0, 'party1': 'B03', 'qty': 1, 'party2': 'S30', 'type': 'Trade', 'p1_qid': 48, 'time': 6.550000000000059, 'price': 105}]
Ask  leg 0  lifts best  Bid 95
counterparty B12 price 102
[{'p2_qid': 61.0, 'party1': 'B12', 'qty': 1, 'party2': 'S17', 'type': 'Trade', 'p1_qid': 58, 'time': 7.350000000000071, 'price': 102}]
Bid  leg 0  lifts best  Ask 105
count

Ask  leg 0  lifts best  Bid 109
counterparty B08 price 109
[{'p2_qid': 917.0, 'party1': 'B08', 'qty': 1, 'party2': 'S07', 'type': 'Trade', 'p1_qid': 914, 'time': 46.62500000000204, 'price': 109}]
Ask  leg 0  lifts best  Bid 103
counterparty B17 price 108
[{'p2_qid': 956.0, 'party1': 'B17', 'qty': 1, 'party2': 'S16', 'type': 'Trade', 'p1_qid': 934, 'time': 47.60000000000226, 'price': 108}]
Ask  leg 0  lifts best  Bid 99
counterparty B05 price 108
[{'p2_qid': 983.0, 'party1': 'B05', 'qty': 1, 'party2': 'S01', 'type': 'Trade', 'p1_qid': 957, 'time': 48.77500000000253, 'price': 108}]
Ask  leg 0  lifts best  Bid 109
counterparty B02 price 109
[{'p2_qid': 999.0, 'party1': 'B02', 'qty': 1, 'party2': 'S26', 'type': 'Trade', 'p1_qid': 989, 'time': 49.26250000000264, 'price': 109}]
Ask  leg 0  lifts best  Bid 99
counterparty B10 price 108
[{'p2_qid': 1000.0, 'party1': 'B10', 'qty': 1, 'party2': 'S05', 'type': 'Trade', 'p1_qid': 984, 'time': 49.27500000000264, 'price': 108}]
Ask  leg 0  lifts bes

expid                    True
time                     True
ZIC       balance_sum    True
          n              True
          pc             True
GVWY      balance_sum    True
          n              True
          pc             True
SHVR      balance_sum    True
          n              True
          pc             True
ZIP       balance_sum    True
          n              True
          pc             True
best_bid                 True
best_ask                 True
dtype: bool

This function checks that the two replay_vars attributes are the same and returns the difference if it exists:
    

In [5]:
identical_replay_vars(sess,sess1)

no differences found


{}

For a more forensic approach to difference checking, this function will look for differences at the end of every period and stop if one occurs.

In [7]:
sess=Market_session(**environ_dic)
sess.process_order=sess.exchange.process_order3w
sess1=copy.deepcopy(sess)
side_by_side_period_by_period_difference_checker(sess,sess1)


Bid  leg 0  lifts best  Ask 105
counterparty S23 price 102
[{'p2_qid': 60.0, 'party1': 'S23', 'qty': 1, 'party2': 'B05', 'type': 'Trade', 'p1_qid': 59, 'time': 7.2625000000000695, 'price': 102}]
Bid  leg 0  lifts best  Ask 105
counterparty S23 price 102
[{'p2_qid': 60.0, 'party1': 'S23', 'qty': 1, 'party2': 'B05', 'type': 'Trade', 'p1_qid': 59, 'time': 7.2625000000000695, 'price': 102}]
Bid  leg 0  lifts best  Ask 105
counterparty S08 price 103
[{'p2_qid': 61.0, 'party1': 'S08', 'qty': 1, 'party2': 'B19', 'type': 'Trade', 'p1_qid': 58, 'time': 7.28750000000007, 'price': 103}]
Bid  leg 0  lifts best  Ask 105
counterparty S08 price 103
[{'p2_qid': 61.0, 'party1': 'S08', 'qty': 1, 'party2': 'B19', 'type': 'Trade', 'p1_qid': 58, 'time': 7.28750000000007, 'price': 103}]
Bid  leg 0  lifts best  Ask 106
counterparty S21 price 103
[{'p2_qid': 79.0, 'party1': 'S21', 'qty': 1, 'party2': 'B03', 'type': 'Trade', 'p1_qid': 62, 'time': 8.575000000000047, 'price': 103}]
Bid  leg 0  lifts best  Ask 10

counterparty S00 price 97
[{'p2_qid': 382.0, 'party1': 'S00', 'qty': 1, 'party2': 'B25', 'type': 'Trade', 'p1_qid': 351, 'time': 27.89999999999895, 'price': 97}]
Bid  leg 0  lifts best  Ask 107
counterparty S00 price 97
[{'p2_qid': 382.0, 'party1': 'S00', 'qty': 1, 'party2': 'B25', 'type': 'Trade', 'p1_qid': 351, 'time': 27.89999999999895, 'price': 97}]
Bid  leg 0  lifts best  Ask 97
counterparty S20 price 97
[{'p2_qid': 393.0, 'party1': 'S20', 'qty': 1, 'party2': 'B02', 'type': 'Trade', 'p1_qid': 378, 'time': 28.387499999998923, 'price': 97}]
Bid  leg 0  lifts best  Ask 97
counterparty S20 price 97
[{'p2_qid': 393.0, 'party1': 'S20', 'qty': 1, 'party2': 'B02', 'type': 'Trade', 'p1_qid': 378, 'time': 28.387499999998923, 'price': 97}]
Bid  leg 0  lifts best  Ask 107
counterparty S03 price 97
[{'p2_qid': 406.0, 'party1': 'S03', 'qty': 1, 'party2': 'B21', 'type': 'Trade', 'p1_qid': 391, 'time': 28.97499999999889, 'price': 97}]
Bid  leg 0  lifts best  Ask 107
counterparty S03 price 97
[{'p

Now make one session object use the process_order and one the new process_order

In [8]:
sess=Market_session(**environ_dic)

sess1=copy.deepcopy(sess)
sess.process_order=sess.exchange.process_order3w
side_by_side_period_by_period_difference_checker(sess,sess1)

Bid  leg 0  lifts best  Ask 105
counterparty S19 price 95
[{'p2_qid': 44.0, 'party1': 'S19', 'qty': 1, 'party2': 'B14', 'type': 'Trade', 'p1_qid': 20, 'time': 6.262500000000055, 'price': 95}]
[{'party2': 'B14', 'qty': 1, 'party1': 'S19', 'type': 'Trade', 'time': 6.262500000000055, 'price': 95}]
Bid  leg 0  lifts best  Ask 95
counterparty S08 price 95
[{'p2_qid': 74.0, 'party1': 'S08', 'qty': 1, 'party2': 'B27', 'type': 'Trade', 'p1_qid': 23, 'time': 7.625000000000075, 'price': 95}]
[{'party2': 'B27', 'qty': 1, 'party1': 'S08', 'type': 'Trade', 'time': 7.625000000000075, 'price': 95}]
Bid  leg 0  lifts best  Ask 95
counterparty S30 price 95
[{'p2_qid': 87.0, 'party1': 'S30', 'qty': 1, 'party2': 'B10', 'type': 'Trade', 'p1_qid': 47, 'time': 8.200000000000069, 'price': 95}]
[{'party2': 'B10', 'qty': 1, 'party1': 'S30', 'type': 'Trade', 'time': 8.200000000000069, 'price': 95}]
Bid  leg 0  lifts best  Ask 95
counterparty S17 price 95
[{'p2_qid': 93.0, 'party1': 'S17', 'qty': 1, 'party2': 'B

Bid  leg 0  lifts best  Ask 107
counterparty S04 price 107
[{'p2_qid': 890.0, 'party1': 'S04', 'qty': 1, 'party2': 'B12', 'type': 'Trade', 'p1_qid': 834, 'time': 42.9375000000012, 'price': 107}]
[{'party2': 'B12', 'qty': 1, 'party1': 'S04', 'type': 'Trade', 'time': 42.9375000000012, 'price': 107}]
Ask  leg 0  lifts best  Bid 98
counterparty B32 price 106
[{'p2_qid': 896.0, 'party1': 'B32', 'qty': 1, 'party2': 'S20', 'type': 'Trade', 'p1_qid': 876, 'time': 43.137500000001246, 'price': 106}]
[{'party2': 'S20', 'qty': 1, 'party1': 'B32', 'type': 'Trade', 'time': 43.137500000001246, 'price': 106}]
Ask  leg 0  lifts best  Bid 98
counterparty B26 price 105
[{'p2_qid': 898.0, 'party1': 'B26', 'qty': 1, 'party2': 'S01', 'type': 'Trade', 'p1_qid': 822, 'time': 43.175000000001255, 'price': 105}]
[{'party2': 'S01', 'qty': 1, 'party1': 'B26', 'type': 'Trade', 'time': 43.175000000001255, 'price': 105}]
Ask  leg 0  lifts best  Bid 98
counterparty B16 price 105
[{'p2_qid': 902.0, 'party1': 'B16', 'qt

In [9]:
identical_replay_vars(sess,sess1)

no differences found


{}

And for completion, swap it over so the new process_order works in replay mode:

In [10]:
sess=Market_session(**environ_dic)
sess.verbose=True
sess1=copy.deepcopy(sess)
sess1.process_order=sess1.exchange.process_order3w
side_by_side_period_by_period_difference_checker(sess,sess1)


trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial

replay False SHVR   0.0 []
Trader Quote: [B07 Bid P=105 Q=1 T= 3.79 QID:-3]
Trader Quote: [B07 Bid P=105 Q=1 T= 7.71 QID:45]

trial0001;  
replay True SHVR   0.0 []
Trader Quote: [B07 Bid P=105 Q=1 T= 3.79 QID:-3]
Trader Quote: [B07 Bid P=105 Q=1 T= 7.71 QID:45]

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  
replay False SHVR   0.0 []
Trader Quote: [S09 Ask P=095 Q=1 T= 7.36 QID:-3]
Trader Quote: [S09 Ask P=123 Q=1 T= 7.78 QID:46]

trial0001;  
replay True SHVR   0.0 []
Trader Quote: [S09 Ask P=095 Q=1 T= 7.36 QID:-3]
Trader Quote: [S09 Ask P=123 Q=1 T= 7.78 QID:46]

trial0001;  
replay False SHVR   0.0 []
Trader Quote: [B12 Bid P=105 Q=1 T= 2.44 QID:-3]
Trader Quote: [B12 Bid P=105 Q=1 T= 7.79 QID:47]

trial0001;  
replay True SHVR   0.0 []
Trader Quote: [B12 Bid P=105 Q=1 T= 2.44 QID:-3]
Trader Quote: [B12 Bid P=105 Q=1 T= 7.79 QID:47]

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  



trial0001;  
replay True ZIP   0.0 []
Trader Quote: [B05 Bid P=105 Q=1 T= 6.53 QID:-3]
Trader Quote: [B05 Bid P=082 Q=1 T=13.11 QID:118]

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  
replay False ZIC   0.0 []
Trader Quote: [S13 Ask P=095 Q=1 T= 1.23 QID:-3]
Trader Quote: [S13 Ask P=267 Q=1 T=13.15 QID:119]

trial0001;  
replay True ZIC   0.0 []
Trader Quote: [S13 Ask P=095 Q=1 T= 1.23 QID:-3]
Trader Quote: [S13 Ask P=267 Q=1 T=13.15 QID:119]

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  
replay False ZIC   0.0 []
Trader Quote: [B35 Bid P=105 Q=1 T= 6.14 QID:-3]
Trader Quote: [B35 Bid P=010 Q=1 T=13.19 QID:120]

trial0001;  
replay True ZIC   0.0 []
Trader Quote: [B35 Bid P=105 Q=1 T= 6.14 QID:-3]
Trader Quote: [B35 Bid P=010 Q=1 T=13.19 QID:120]

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  




trial0001;  
replay True ZIP   0.0 []
Trader Quote: [B05 Bid P=105 Q=1 T= 6.53 QID:-3]
Trader Quote: [B05 Bid P=101 Q=1 T=19.66 QID:197]

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  
replay False ZIP   0.0 []
Trader Quote: [S00 Ask P=096 Q=1 T=19.09 QID:-3]
Trader Quote: [S00 Ask P=124 Q=1 T=19.79 QID:198]

trial0001;  
replay True ZIP   0.0 []
Trader Quote: [S00 Ask P=096 Q=1 T=19.09 QID:-3]
Trader Quote: [S00 Ask P=124 Q=1 T=19.79 QID:198]

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  
replay False SHVR   0.0 []
Trader Quote: [S10 Ask P=096 Q=1 T=19.71 QID:-3]
Trader Quote: [S10 Ask P=104 Q=1 T=19.85 QID:199]
[{'party2': 'S10', 'qty': 1, 'party1': 'B09', 'type': 'Trade', 'time': 19.8499999


trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  
replay False ZIC   0.0 []
Trader Quote: [S07 Ask P=097 Q=1 T=21.52 QID:-3]
Trader Quote: [S07 Ask P=374 Q=1 T=23.97 QID:277]

trial0001;  
replay True ZIC   0.0 []
Trader Quote: [S07 Ask P=097 Q=1 T=21.52 QID:-3]
Trader Quote: [S07 Ask P=374 Q=1 T=23.97 QID:277]

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  
replay False ZIP   0.0 []
Trader Quote: [B36 Bid P=106 Q=1 T=20.86 QID:-3]
Trader Quote: [B36 Bid P=092 Q=1 T=24.01 QID:278]

trial0001;  
replay True ZIP   0.0 []
Trader Quote: [B36 Bid P=106 Q=1 T=20.86 QID:-3]
Trader Quote: [B36 Bid P=092 Q=1 T=24.01 QID:278]

trial0001;  

trial0001;  

trial0001;  
replay False SHVR   0.0 []
Trader Quote: [S04 Ask P=097 Q=1 T=23.34 QID:-3]
Trader Quote: [S04 Ask P=110 Q=1 T

...


trial0001;  
replay False ZIC   0.0 []
Trader Quote: [S17 Ask P=097 Q=1 T=30.00 QID:-3]
Trader Quote: [S17 Ask P=843 Q=1 T=31.45 QID:437]

trial0001;  
replay True ZIC   0.0 []
Trader Quote: [S17 Ask P=097 Q=1 T=30.00 QID:-3]
Trader Quote: [S17 Ask P=843 Q=1 T=31.45 QID:437]

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  
replay False ZIC   0.0 []
Trader Quote: [S02 Ask P=097 Q=1 T=28.36 QID:-3]
Trader Quote: [S02 Ask P=511 Q=1 T=31.49 QID:438]

trial0001;  
replay True ZIC   0.0 []
Trader Quote: [S02 Ask P=097 Q=1 T=28.36 QID:-3]
Trader Quote: [S02 Ask P=511 Q=1 T=31.49 QID:438]

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  
killing lastquote=[S24 Ask P=104 Q=1 T=13.36 QID:121]

trial0001;  
killing lastquote=[S24 Ask P=104 Q=1 T=13.36 QID:121]

trial0001;  

trial0001;  

trial0001;  
replay False ZIC   0.0 []
Trader Quote: [S02 Ask P=097 Q=1 T=28.36 QID:-3]
Trader Quote: [S02 Ask P=301 Q=1 T=31.55 QID:439]

trial0001;  
replay True ZIC  

replay True ZIC   0.0 []
Trader Quote: [S01 Ask P=097 Q=1 T=21.67 QID:-3]
Trader Quote: [S01 Ask P=499 Q=1 T=35.42 QID:524]

trial0001;  
replay False ZIP   0.0 []
Trader Quote: [B15 Bid P=106 Q=1 T=15.35 QID:-3]
Trader Quote: [B15 Bid P=105 Q=1 T=35.44 QID:525]

trial0001;  
replay True ZIP   0.0 []
Trader Quote: [B15 Bid P=106 Q=1 T=15.35 QID:-3]
Trader Quote: [B15 Bid P=105 Q=1 T=35.44 QID:525]

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  
replay False ZIC   0.0 []
Trader Quote: [S13 Ask P=095 Q=1 T= 1.23 QID:-3]
Trader Quote: [S13 Ask P=186 Q=1 T=35.47 QID:526]

trial0001;  
replay True ZIC   0.0 []
Trader Quote: [S13 Ask P=095 Q=1 T= 1.23 QID:-3]
Trader Quote: [S13 Ask P=186 Q=1 T=35.47 QID:526]

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  
replay False ZIC   0.0 []
Trader Quote: [S20 Ask P=097 Q=1 T=30.98 QID:-3]
Trader Quote: [S20 Ask P=630 Q=1 T=35.52 QID:527]

trial0001;  
replay True ZIC   0.0 []
Trad

replay False SHVR   5.0 [{'party2': 'B25', 'qty': 1, 'party1': 'S28', 'type': 'Trade', 'time': 28.58749999999891, 'price': 102}]
Trader Quote: [S28 Ask P=098 Q=1 T=34.98 QID:-3]
Trader Quote: [S28 Ask P=108 Q=1 T=39.19 QID:639]
[{'party2': 'S28', 'qty': 1, 'party1': 'B11', 'type': 'Trade', 'time': 39.18750000000035, 'price': 108}]

trial0001;  
replay True SHVR   5.0 [{'p2_qid': 370.0, 'party1': 'S28', 'qty': 1, 'party2': 'B25', 'type': 'Trade', 'p1_qid': 368, 'time': 28.58749999999891, 'price': 102}]
Trader Quote: [S28 Ask P=098 Q=1 T=34.98 QID:-3]
Trader Quote: [S28 Ask P=108 Q=1 T=39.19 QID:639]
Ask  leg 0  lifts best  Bid 108
counterparty B11 price 108
[{'p2_qid': 639.0, 'party1': 'B11', 'qty': 1, 'party2': 'S28', 'type': 'Trade', 'p1_qid': 615, 'time': 39.18750000000035, 'price': 108}]

trial0001;  
replay False ZIC   0.0 []
Trader Quote: [B23 Bid P=107 Q=1 T=24.24 QID:-3]
Trader Quote: [B23 Bid P=052 Q=1 T=39.20 QID:640]

trial0001;  
replay True ZIC   0.0 []
Trader Quote: [B23 B

replay True GVWY   6.0 [{'p2_qid': 272.0, 'party1': 'B16', 'qty': 1, 'party2': 'S16', 'type': 'Trade', 'p1_qid': 223, 'time': 23.5249999999992, 'price': 103}]
Trader Quote: [S16 Ask P=098 Q=1 T=42.80 QID:-3]
Trader Quote: [S16 Ask P=098 Q=1 T=44.04 QID:738]
Ask  leg 0  lifts best  Bid 98
counterparty B33 price 105
[{'p2_qid': 738.0, 'party1': 'B33', 'qty': 1, 'party2': 'S16', 'type': 'Trade', 'p1_qid': 721, 'time': 44.03750000000145, 'price': 105}]

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  
replay False SHVR   4.0 [{'party2': 'S26', 'qty': 1, 'party1': 'B36', 'type': 'Trade', 'time': 28.7749999999989, 'price': 101}]
Trader Quote: [S26 Ask P=098 Q=1 T=40.33 QID:-3]
Trader Quote: [S26 Ask P=105 Q=1 T=44.11 QID:739]

trial0001;  
replay True SHVR   4.0 [{'p2_qid': 376.0, 'party1': 'B36', 'qty': 1, 'party2': 'S26', 'type': 'Trade', 'p1_qid': 343, 'time': 28.7749999999989, 'price':


trial0001;  
replay True ZIC   0.0 []
Trader Quote: [S01 Ask P=097 Q=1 T=21.67 QID:-3]
Trader Quote: [S01 Ask P=213 Q=1 T=48.13 QID:819]

trial0001;  
replay False ZIC   0.0 []
Trader Quote: [S31 Ask P=098 Q=1 T=35.68 QID:-3]
Trader Quote: [S31 Ask P=465 Q=1 T=48.14 QID:820]

trial0001;  
replay True ZIC   0.0 []
Trader Quote: [S31 Ask P=098 Q=1 T=35.68 QID:-3]
Trader Quote: [S31 Ask P=465 Q=1 T=48.14 QID:820]

trial0001;  
killing lastquote=[S05 Ask P=106 Q=1 T=13.64 QID:128]

trial0001;  
killing lastquote=[S05 Ask P=106 Q=1 T=13.64 QID:128]

trial0001;  

trial0001;  

trial0001;  
replay False ZIC   0.0 []
Trader Quote: [S27 Ask P=097 Q=1 T=32.00 QID:-3]
Trader Quote: [S27 Ask P=404 Q=1 T=48.18 QID:821]

trial0001;  
replay True ZIC   0.0 []
Trader Quote: [S27 Ask P=097 Q=1 T=32.00 QID:-3]
Trader Quote: [S27 Ask P=404 Q=1 T=48.18 QID:821]

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  
replay False ZIC   0.0 []
Trader Quote: [S17 Ask P=099 Q=1 T=46.77 QID:-3]